In [1188]:
from pathlib import Path

import pandas as pd

In [1189]:
raw = None
for name in Path("/tmp/actor_mesh_benchmark/").glob("*.csv"):
    print(name)
    if raw is None:
        raw = pd.read_csv(name)
    else:
        raw = pd.concat([raw, pd.read_csv(name)])

raw.head()

/tmp/actor_mesh_benchmark/d47ab06074d4.csv
/tmp/actor_mesh_benchmark/83ef35e1bd39.csv
/tmp/actor_mesh_benchmark/34d5605c7607.csv
/tmp/actor_mesh_benchmark/678349f91d72.csv
/tmp/actor_mesh_benchmark/b270761e9a45+.csv


,test_name,rev,rev_name,hosts,gpus,message_size,bytes,casts,messages,total_test_duration_us,total_test_iterations,avg_duration_us,min_duration_us,max_duration_us,stddev_duration_us
0,ActorScaling,d47ab06074d4,[monarch] Small refa,1,1,10,40470.0,4047.0,4047.0,3.004358e+06,4046,742.550113,564.931892,4591.648001,62.697749
1,ActorScalingMultipart,d47ab06074d4,[monarch] Small refa,1,1,10,41250.0,4125.0,4125.0,3.059318e+06,4124,741.832634,609.939918,3722.568974,54.503967
2,ActorScaling,d47ab06074d4,[monarch] Small refa,1,1,100,412300.0,4123.0,4123.0,3.010706e+06,4122,730.399309,587.394927,3103.805706,50.538460
3,ActorScalingMultipart,d47ab06074d4,[monarch] Small refa,1,1,100,479800.0,4798.0,4798.0,3.551600e+06,4797,740.379352,593.344215,3040.631767,48.266389
4,ActorScaling,d47ab06074d4,[monarch] Small refa,1,1,1000,4502000.0,4502.0,4502.0,3.270353e+06,4501,726.583730,588.917173,3181.774169,49.756583


In [1191]:
raw["bps"] = raw["bytes"] / (raw["total_test_duration_us"] / 1_000_000)
raw["mps"] = raw["messages"] / (raw["total_test_duration_us"] / 1_000_000)
raw["cps"] = raw["casts"] / (raw["total_test_duration_us"] / 1_000_000)
raw["test_id"] = raw["test_name"] + "_" + raw["rev_name"] + "_" + raw["rev"]

In [1192]:
import plotly.express as px

In [1193]:
px.line(
    raw,
    x="message_size",
    y="cps",
    log_y=False,
    color="test_id",
    labels={},
    log_x=False,
    line_group="gpus",
)

In [1194]:
# Group by message_size and find minimum values for each field
min_values = (
    raw.groupby("message_size")
    .agg({"bps": "min", "mps": "min", "cps": "min", "total_test_duration_us": "min"})
    .reset_index()
)

# Rename columns to indicate they are minimum values
min_values.columns = ["message_size"] + [
    f"min_{col}" for col in min_values.columns if col != "message_size"
]

# Merge the minimum values back to the original dataframe
mps_ratio = pd.merge(raw, min_values, on="message_size")

# Create normalized fields
mps_ratio["bps_norm"] = mps_ratio["bps"] / mps_ratio["min_bps"]
mps_ratio["mps_norm"] = mps_ratio["mps"] / mps_ratio["min_mps"]
mps_ratio["cps_norm"] = mps_ratio["cps"] / mps_ratio["min_cps"]
mps_ratio["duration_norm"] = (
    mps_ratio["total_test_duration_us"] / mps_ratio["min_total_test_duration_us"]
)

In [1196]:
px.line(
    mps_ratio,
    x="message_size",
    y="mps_norm",
    facet_row="gpus",
    color="test_id",
    labels={"mps_norm": "Messages/s vs. Baseline"},
    log_x=True,
)